# Active (sequential) learning for experimental band gap prediction

## Overview
### Context
The band gap of a material dictates many of its electrical properties and defines whether or not it is an insulator, a semiconductor, or a conductor. A model that can predict the experimental band gap of a given material, as well as guide/screen new materials is useful in a variety of electronic applications.
### Problem formulation
Develop a machine learning (ML) model that can predict the experimental band gap of a material given only its composition. Utilize active learning to guide the exploration of design space to optimize the band gap for a certain application. Here, we will seek to maximize the band gap (e.g., identify insulators), but one could use the same approach to minimize or optimize to a certain energy range.
## Approach
### 1. Data set importing.
- Will use the `matbench_expt_gap` dataset
    - For the sake of time, I filtered out all materials with a band gap of zero. Ideally, I would develop a classification model to identify metals as an initial pre-screening step before performing regression. Then for prediction on a new material, metals could automatically be assigned a band gap of zero and all others could use the trained regression model.
- For now, will only use compositional features
- See `eda.ipynb` for details regarding data cleaning and exploratory data analysis
     
### 2. As the output (i.e., target variable) is continuous, we will use regression.
- I chose random forest because it is relatively robust, especially for smaller data sets (relative to neural networks)
- Hyperparameter tuning and feature selection are done with `sklearn`
- Final model training and prediction is done with `lolopy` because its random forest regressor has an option to return the standard deviation of the predictions from all trees (needed for two of the acquisition functions)
     
### 3. For active learning, we will test several acquisition functions for each step of training data additions.
- MEI (Maximum Expected Improvement), MLI (Maximum Likelihood of Improvement) and MU (Maximum Uncertainty)
    - Uncertainty estimates for random forest can be found [here](https://jmlr.org/papers/volume15/wager14a/wager14a.pdf).

# Import modules 

In [1]:
# General python 
from scipy import stats
import pandas as pd
import numpy as np

# Plotting
import helpers.plotting as my_plt

# Machine learning training & prediction
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.inspection import permutation_importance
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor as skl_RandomForestRegressor
from lolopy.learners import RandomForestRegressor

# Model persistence
from joblib import dump, load

# To ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Hyperparameter tuning and feature selection
def tune(X_df, y_ser, in_train = 'all', outf = 'grid_search_trained_model'):
    """
    Performs an initial round of hyperparameter tuning with all features, then feature selection,
    and finally a final round of hyperparameter tuning with top features from feature selection.

    Parameters
    ----------
    X_df : pandas dataframe 
            Input data
    y_ser : pandas series
            Target data
    in_train: 'all' or list-like
            Rows to include in training set

    Returns
    -------
    best_params : dict
            Tuned hyperparameters
    best_estimator : sklearn estimator
            Tuned model
    top_feat : list-like
            Top features from feature selection
    outf : str
            Prefix for joblib file containing tuned model
    """
    X = np.array(X_df)
    y = np.array(y_ser)
    if in_train == 'all':
        in_train = list(range(len(y)))
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
    else:
        X_train = X[in_train]
        y_train = y[in_train]
        X_test = X[~in_train]
        y_test = y[~in_train]
    
    # Hyperparameters to tune - here I only tune the number of trees and the maximum
    # depth because these are the two primary common hyperparameters between
    # sklearn's and lolopy's random forest regressors
    dense_grid = {'n_estimators': [15, 20, 50, 100, 200],
                  'max_depth': [5,10,20,50,100,1073741824]
    }
    # Hyperparameter tuning with sklearn
    model = skl_RandomForestRegressor()
    grid = GridSearchCV(estimator = model, param_grid = dense_grid, cv = 3, verbose=2,  n_jobs = -1)
    grid.fit(X_train, y_train)
    best_params = grid.best_params_
    best_estimator = grid.best_estimator_

    # Feature selection
    result = permutation_importance(
        best_estimator, X_test, y_test, scoring = "r2", n_repeats=10, random_state=42
    )
    forest_importances = pd.Series(result.importances_mean, index=X_df.columns)
    std = result.importances_std
    sorted_ind = forest_importances.argsort()[::-1]
    sorted_importances = forest_importances.sort_values(ascending=False)
    if in_train == 'all':
        top_feat = sorted_importances[ sorted_importances > 0.0 ].index
    else:
        top_feat = sorted_importances[ :10 ].index
    X = np.array(df[top_feat])

    # Final round of hyperparameter tuning
    tmp = grid.fit(X_train, y_train)
    best_params = grid.best_params_
    best_estimator = grid.best_estimator_
    # Save tuned model
    dump(best_estimator, '{}.joblib'.format(outf))
    return best_params, best_estimator, top_feat

In [3]:
# Active learning: function to update training set
def update(X, y_pred, y_std, train_inds, acq = 'mei', n = 5):
    all_inds = set(range(len(X))) # indices for entire dataset
    search_inds = list(all_inds.difference(train_inds)) # test set indices
    acq = acq.lower()
    if acq == 'mei':
        return train_inds + [search_inds[s] for s in y_pred.argsort()[::-1][:n]]
    elif acq == 'mli':
        return train_inds + [search_inds[s] for s in (np.divide(y_pred-np.max(y[train_inds]),y_std)).argsort()[::-1][:n]]
    elif acq == 'mu':
        return train_inds + [search_inds[s] for s in y_std.argsort()[::-1][:n]]
    else:
        return train_inds + [s for s in np.random.choice(search_inds, n)] 

In [ ]:
# There are some off-diagonals with a correlation of 1, so drop 1 of each of these pairs
X_df.drop(columns = ['MagpieData range NfUnfilled', 'MagpieData minimum NsUnfilled'], inplace = True)

## Skewness
Here, I check for skewness of the features and target. Some of the features are highly skewed due to low variance (i.e., most of the samples take on one value) - I chose to remove these from the data set. The target is also skewed and becomes more normal after log-transforming - I compare model training/prediction on both the gap and log-transformed gap in the ML section.

In [ ]:
skew_val = X_df.skew(axis = 0)
skewed = skew_val[ abs( skew_val ) > 1.0 ]
skewed.sort_values()

In [ ]:
# Use threshold of 7.0 to identify highly skewed variables and plot their distributions
to_plot = skewed[ abs(skewed.sort_values()) > 7.0 ].index
boxplot(X_df, to_plot, nrows = 2)
distplot(X_df, to_plot, nrows = 2)

In [ ]:
# Drop features with low variance
for feature in to_plot:
    this_var = df[feature].var()
    print('{}: {}'.format(feature, this_var))
    if this_var < 0.1:
        X_df.drop(columns = [feature], inplace = True)

### Log transformations
Does not help features, but helps target, so I only include the log-transformed target in the final dataframe.

In [ ]:
X_df['log_MagpieData mode NfValence'] = np.log(X_df['MagpieData mode NfValence']+1)
boxplot(X_df, ['log_MagpieData mode NfValence'], nrows = 1, figsize = (5,5))
distplot(X_df, ['log_MagpieData mode NfValence'], nrows = 1, figsize = (5,5))
X_df.drop(columns = ['MagpieData mode NfValence','log_MagpieData mode NfValence'],inplace = True)

In [ ]:
X_df['log_MagpieData mode NdUnfilled'] = np.log(X_df['MagpieData mode NdUnfilled']+1)
boxplot(X_df, ['log_MagpieData mode NdUnfilled'], nrows = 1, figsize = (5,5))
distplot(X_df, ['log_MagpieData mode NdUnfilled'], nrows = 1, figsize = (5,5))
X_df.drop(columns = ['MagpieData mode NdUnfilled','log_MagpieData mode NdUnfilled'],inplace = True)

In [ ]:
# Check target
y.skew(axis = 0)

In [ ]:
boxplot(df, ['gap expt'], nrows = 1, figsize = (5,5))
distplot(df, ['gap expt'], nrows = 1, figsize = (5,5))

In [ ]:
df['log_gap expt'] = np.log(y+1)
boxplot(df, ['log_gap expt'], nrows = 1, figsize = (5,5))
distplot(df, ['log_gap expt'], nrows = 1, figsize = (5,5))

## Principal component analysis
Here, I perform PCA to see if the data is clustered. As it is not, no further clustering analysis is performed.

In [ ]:
# Scale data (necessary for PCA!)
scaler = StandardScaler()
X_scaled = pd.DataFrame( scaler.fit_transform(X_df), columns = X_df.columns )
# Start with 3 components
pca = PCA(n_components=3)
pca.fit(X_scaled)
print(pca.explained_variance_ratio_)
X_pca = pca.transform(X_scaled)

In [ ]:
fig = plt.figure(1, figsize=(10, 5))
ax = fig.add_subplot(111, projection="3d", elev=20, azim=134)
ax.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:, 2], c=y, cmap='viridis')

In [ ]:
# Now check in two dimensions
pca = PCA(n_components=2)
pca.fit(X_scaled)
print(pca.explained_variance_ratio_)
X_pca = pca.transform(X_scaled)
fig,ax = plt.subplots()
ax.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='viridis')

# Full model training & Prediction
Here, I perform model training and prediction on the entire data set and evaluate the performance of each model with 5-fold cross-validation. I test the effect of log-transforming the gap, as well as removing outliers in the gap.

In [ ]:
# First tune without log transformation of target
params, model, top_feat = tune(X_scaled, y, outf = 'full_model')
cross_val_score(model, X_scaled[top_feat], y, scoring = 'neg_mean_absolute_error', cv = 5)

In [ ]:
# Next tune with log transformation of target
params_log, model_log, top_feat_log = tune(X_scaled, np.log(y), outf = 'full_model_log')
cross_val_score(model, X_scaled[top_feat_log], np.log(y), scoring = 'neg_mean_absolute_error', cv = 5)

In [ ]:
# Next tune remove outliers in target
outliers_inds = np.abs(stats.zscore(y)) < 3
y_no_out = y[outliers_inds]
X_no_out = X_scaled.drop(outliers_inds[(outliers_inds == False)].index)
params_no_out, model_no_out, top_feat_no_out = tune(X_no_out, np.log(y_no_out), outf = 'full_model_log_noout')
cross_val_score(model, X_no_out[top_feat_no_out], np.log(y_no_out), scoring = 'neg_mean_absolute_error', cv = 5)

In [ ]:
# Predict with no log and with outliers
X_train, X_test, y_train, y_test = train_test_split(X_df[top_feat], y, random_state = 0)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

In [ ]:
# Predict with log and with outliers
X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(X_df[top_feat_log], np.log(y), random_state = 0)
model_log.fit(X_train_log,y_train_log)
y_pred_log = model_log.predict(X_test_log)

In [ ]:
# Predict with log and no outliers
X_train_no_out, X_test_no_out, y_train_no_out, y_test_no_out = train_test_split(X_no_out[top_feat_no_out], 
                                                                                np.log(y_no_out), random_state = 0)
model_no_out.fit(X_train_no_out,y_train_no_out)
y_pred_no_out = model_no_out.predict(X_test_no_out)

In [ ]:
predvtrue(y_test.to_numpy(), y_pred)
print(r2_score(y_test, y_pred))

In [ ]:
predvtrue(np.exp(y_test_log.to_numpy()), np.exp(y_pred_log))
print(r2_score(y_test_log, y_pred_log))

In [ ]:
predvtrue(np.exp(y_test_no_out.to_numpy()), np.exp(y_pred_no_out))
print(r2_score(y_test_no_out, y_pred_no_out))

### Observations
All models have similar $R^2$ but vary slightly in $MAE$. Further improvements could be made with more extensive feature engineering, hyperparameter tuning, and aggregation of predictions from different models.

# Active learning

The goal of this section is to guide the exploration of design space to identify compositions with a high band gap. In a real life scenario, we could use active learning to choose which experiments or simulations to prioritize. 

We start initially with 10 data points, then incrementally add 5 at each active learning step.

In [ ]:
# Select initial training set
np.random.seed(100)

in_train = np.zeros(len(X_scaled), dtype=bool)
in_train[np.random.choice(len(X_scaled), 10, replace=False)] = True
assert not np.isclose(max(y), max(y[in_train]))

In [ ]:
# Initial hyperparameter tuning and feature selection. Here, I only do this once, but ideally would do at 
# every step or every number of steps. Performing feature selection is important here for avoiding the curse
# of dimensionality (having many more features than samples).
all_inds = set(range(len(y)))
train = [np.where(in_train)[0].tolist()]
train_inds = train[-1].copy()    # Initial Set
search_inds = list(all_inds.difference(train_inds)) # All samples not in the current set
# Hyperparameter tuning and feature selection
params, model, top_feat = tune(X_scaled, y, np.array(train_inds), outf = 'active_model')
X_top = np.array(X_scaled[top_feat])
# Model training and prediction with tuned hyperparameters and features
model = RandomForestRegressor(num_trees = params['n_estimators'], max_depth = params['max_depth'])
model.fit(X_top[train_inds], y[train_inds])

In [ ]:
%%capture
# Active learning steps
n_steps = 50
mae = np.zeros((4,n_steps))
y_max = np.zeros((4,n_steps))
for a,acq in enumerate(['random', 'mei', 'mli', 'mu']):
    train = [np.where(in_train)[0].tolist()]
    for i in range(n_steps):
        train_inds = train[-1].copy()    # Initial Set
        search_inds = list(all_inds.difference(train_inds)) # All samples not in the current set
        model.fit(X_top[train_inds], y[train_inds])
        y_pred, y_std = model.predict(X_top[search_inds], return_std = True) # Predictions
        mae[a,i] = mean_absolute_error(y[search_inds], y_pred)
        y_max[a,i] = max( y[train_inds] )
        # Update training set
        train_inds = update(X_scaled, y_pred, y_std, train_inds, acq=acq)
        train.append(train_inds) # Storage of the current set per step

In [ ]:
# Compare ability of each acquisition function to find materials with high gaps
true_max = max(y)
random_max = y_max[0,:]
mei_max = y_max[1,:]
mli_max = y_max[2,:]
mu_max = y_max[3,:]
fig,ax = plt.subplots()
ax.plot(range(n_steps), [true_max]*n_steps, 'k', linestyle = 'dashed')
ax.plot(range(n_steps), random_max, label='Random')
ax.plot(range(n_steps), mei_max, label='MEI')
ax.plot(range(n_steps), mli_max, label='MLI')
ax.plot(range(n_steps), mu_max, label='MU')
ax.legend()

### Observations
All acquisition functions to better than selecting at random and find the material with the maximum gap within 20 steps.

# Conclusions
Conventional machine learning training was used to develop a model that can predict the experimental band gap purely from composition with an $R^2$ of approximately 0.7 and an $MAE$ of approximately 0.15 eV. Active learning was successful in accelerating the exploration of design space to maximize the band gap. Further development could include further feature engineering and testing of different algorithms.